# Fitting of dqdv peaks
The purpose of this notebook is to evaluate and develope a robust way of fitting dqdv data. The plan is then to implement this into the cellpy.utils.ica module (as seperate classes). It would also be valuable to equip the fitting class(es) with optional ipywidgets.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import logging
import bokeh
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cellpy import cellreader
from cellpy.utils import ica
import holoviews as hv

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

%matplotlib inline
hv.extension('bokeh')

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

my_data = cellreader.CellpyData()
filename = "../../../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)
my_data.load(filename)
my_data.set_mass(0.1)

from icafit import *

## Defining dqdv peak ensambles
The natural way (and my impression is that this is how other groups also do it) of conducting an "in-depth" ica study on a LiB cell would be to measure ica of re-buildt half-cells of both the cathode and the anode of the full cells, fit the peaks of the half cells, and then a use convolution of these fits to fit the actual full cell.


### Examples from the literature
Should include references here...

### Plan
1. Create a class (PeakEnsamble)
2. Create peak ensambles by sub-classing PeakEnsamble

### ToDo
- [x] Fix so that it is possible to turn crystalline peak on and off for Si peaks
- [ ] Set peak attributes directly
- [ ] Make it easy (and obvious) to use previous fit-prms for new fit
- [x] Use `__add__`
- [x] Combine all fit-results into one dataframe
- [ ] Make a ipywidget for at least one of the prms (e.g. scale)
- [ ] Make it possible to freeze peaks and ensambles
- [ ] Make it possible to zero out peaks?
- [x] Fit negative of discharge curves
- [ ] Fit discharge and charge in one go? ("hysteresis parameter")

In [ ]:
logger.setLevel(logging.DEBUG)

In [ ]:
import logging
from colorama import Fore
import ipywidgets as widgets

class log_viewer(logging.Handler):
    """ Class to redistribute python logging data """

    # have a class member to store the existing logger
    logger_instance = logging.getLogger("__name__")

    def __init__(self, output=None, up_side_down=True, max_lines=20, *args, **kwargs):
        self._output = output
        self.up_side_down = up_side_down
        self.max_lines = max_lines
        if self._output is None:
            self._output = widgets.Output(layout=widgets.Layout(width='600px', height='160px', border='solid')) 
        self._output.layout.overflow_y = "scroll"
 
        # Initialize the Handler
        logging.Handler.__init__(self, *args)

        # optional take format
        # setFormatter function is derived from logging.Handler
        for key, value in kwargs.items():
            if "{}".format(key) == "format":
                self.setFormatter(value)

        # make the logger send data to this class
        self.logger_instance.addHandler(self)
        
    @property
    def output(self):
        return self._output
        

    def emit(self, record):
        """ Overload of logging.Handler method """

        record = self.format(record)
        
        if self.up_side_down:
            self.output.outputs = (
                {
                    'name': 'stdout', 
                    'output_type': 
                    'stream', 
                    'text': (Fore.BLACK + (record + '\n'))
                },
            ) + self.output.outputs[:self.max_lines]
            
        else:
            self.output.outputs = self.output.outputs[-self.max_lines:] + (
                {
                    'name': 'stdout', 
                    'output_type': 'stream', 
                    'text': (Fore.BLACK + (record + '\n'))
                },
            )
            


## Widgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class SiliconPeaksFitWidget(widgets.VBox):
    def __init__(self, silicon_peaks, x, y, name=None):
        """
        """
        #    [ ]             jitter
        #    [x] |-----o--|  scale
        #    [x] |-----o--|  shift
        #    [x] |-----o--|  sigma_p1
        #    [x] |-----o--|  max_point
        # TODO:
        # - shift is not refined on (since it is only used during _read_peak_definitions)
        #      should implement overall shift of the peaks (but not sure how to do it (maybe use expression?) (or ConstantModel))
        # Need to implement result (and parameters) and update (as default) + button for reset
        # Need to make this work for CompositeEnsamble
        
        
        self.peaks_object = silicon_peaks
        self.x = x
        self.y = y
        self.invert_res = False
        self.invert_dq = False
        
        self.result = None
        
        if name is None:
            name = self.peaks_object.name
        jitter = self.peaks_object.jitter
        scale = self.peaks_object.scale
        shift = self.peaks_object.shift
        sigma_p1 = self.peaks_object.sigma_p1
        max_point = self.peaks_object.max_point
        crystalline = self.peaks_object.crystalline
        compress = self.peaks_object._compress
        expand = self.peaks_object._expand
        
        self.plot_output = widgets.Output()
        self.log_output = widgets.Output()
        self.plot_fixed = False
        
        self.w_name = widgets.Label(f"{name}")
        
        self.w_jitter = widgets.Checkbox(value=jitter, description="jitter")
        self.w_crystalline = widgets.Checkbox(value=crystalline, description="crystalline")
        
        self.w_scale = widgets.FloatSlider(
            value=scale, 
            min=0.00001,
            max=100*scale,
            continuous_update=False,
            description="scale",  # Currently using the description to link up to params (so you have to live with the bad names)
        )
        
        self.w_shift = widgets.FloatSlider(
            value=shift, 
            min = -1,
            max = 1,
            step = 0.01,
            continuous_update=False,
            description="shift",
        )
        
        self.w_sigma_p1 = widgets.FloatSlider(
            value=sigma_p1, 
            min=0.000000001,
            max=10*sigma_p1,
            step=0.01,
            continuous_update=False,
            description="sigma_p1",
        )
        
        self.w_max_point = widgets.FloatSlider(
            value=max_point, 
            min = 0.00001,
            max = 10*max_point,
            continuous_update=False,
            description="max_point",
        )
        
        self.w_max_point = widgets.FloatSlider(
            value=max_point, 
            min = 0.00001,
            max = 10*max_point,
            continuous_update=False,
            description="max_point",
        )
        
        self.w_fit = widgets.Button(
            description="Fit!"
        )
        
        self.clear = widgets.Button(
            description="Clear!"
        )
        
        self.w_fit.on_click(self.fit)
        self.clear.on_click(self.clear_log)
       
        self.w_jitter.observe(self.on_w_change, 'value')
        self.w_scale.observe(self.on_w_change, 'value')
        self.w_shift.observe(self.on_w_change, 'value')
        self.w_sigma_p1.observe(self.on_w_change, 'value')
        self.w_max_point.observe(self.on_w_change, 'value')
        self.w_crystalline.observe(self.on_w_change, 'value')
        
        self.widget_ids = ['jitter', 'scale', 'shift', 'sigma_p1', 'max_point', 'crystalline']

        super(SiliconPeaksFitWidget, self).__init__()
        
        widget_box = widgets.VBox(
            [
                self.w_name,
                self.w_crystalline,
                self.w_jitter,
                self.w_scale,
                self.w_shift,
                self.w_sigma_p1,
                self.w_max_point,
                widgets.HBox([self.w_fit, self.clear]),
            ]
        )
        
        row_1 = widgets.HBox([widget_box, self.plot_output])
        row_2 = self.log_output
        
        self.children = [
            row_1,
            row_2,
        ]
        
        self.update_plot("initial")
        
    def fit(self, change=None, y=None, x=None):
        # TODO: need to implement turning off y and x 
        
        _old_shift = self.peaks_object.shift
        _old_max_point = self.peaks_object.max_point
        _old_scale = self.peaks_object.scale
        
        if y is None:
            y = self.y
            
        if x is None:
            x = self.x
            
        if self.invert_dq:
            y = - y

            
        with self.log_output:
            print("-... ")
            print()
            self.result = self.peaks_object.fit(y, x=x)
            print("--.. - . -- .")
            print("-go----------")
            print("  // o-o \\  ")
            print("   -| u |-   ")
            print("     \./     ")
            print("---------ok-?")
            print("--- - . -- ..")
            display(self.result)
            print(self.peaks_object)
            
        main_logger.info(f"auto update: {self.peaks_object.auto_update_from_fit}")
        _new_shift = self.peaks_object.shift
        _new_max_point = self.peaks_object.max_point
        _new_scale = self.peaks_object.scale
        main_logger.info(f"shift from {_old_shift} to {_new_shift}")
        main_logger.info(f"max_point from {_old_max_point} to {_new_max_point}")
        main_logger.info(f"scale from {_old_scale} to {_new_scale}")
            
        self.update_plot("new_fit")

        self.plot_fixed = True
        for what in self.widget_ids:
            value = getattr(self.peaks_object, what)
            main_logger.info(f" -> update widget. widget: {what} value: {value}")
            w = "w_" + what
            getattr(self, w).value = value
        self.plot_fixed = False
        
         
    def on_w_change(self, change):
        
        name = change.owner.description
        value = change.new
        main_logger.info(f"change observed name: {name} value: {value}")
        setattr(self.peaks_object, name, value)
        self.peaks_object.init()  # This removes the link to from the parameters to the widgets :-(
        self.update_plot(name)
        
    def _create_plot_object(self, components=None, group_title="fit", 
                           invert_dq=False, invert_res=False, width=500, height=500, size=8):

        if self.invert_dq:
            y = - self.y
        else:
            y = self.y

        i = 1
        if self.invert_res:
            i = -1
        main_logger.info("-> creating plot object")
        raw = hv.Points((self.x, y), label="raw", group=group_title).opts(
            width=width, height=height, size=size, alpha=0.3,
            xlabel="Voltage",
            ylabel="dQ/dv",
        )
        if components is not None:
            main_logger.info("-> components are not None")
            prt = {}
            for key in components:
                if not key.endswith("Scale"):
                    prt[key] = hv.Curve((self.x, i * components[key]), group=group_title)
            return raw * hv.NdOverlay(prt)

            
        prt = {
            "init": hv.Curve((self.x, i * self.result.init_fit), group=group_title).opts(alpha=0.5),
            "best": hv.Curve((self.x, i * self.result.best_fit), group=group_title),
        }

        parts = self.result.eval_components(x=self.x)
        if not parts:
            main_logger.info("-> no parts extracted")
        
        s = self.peaks_object.scale  # set this to 1 if you dont want to use the scale factor when plotting
        for key in parts:
            if not key.endswith("Scale"):
                main_logger.info(f"-> adding {key} to the plot (scaled)")
                prt[key] = hv.Curve((self.x, i * s * parts[key]), group=group_title)

        return raw * hv.NdOverlay(prt)

        
    def update_plot(self, name):
        if self.plot_fixed:
            main_logger.info("sorry, plot is fixed")
            return
        
        if name in ["scale", "shift", "sigma_p1", "max_point", "initial", "crystalline", "new_fit"]:
            
            with self.plot_output:
                if name == "new_fit":
                    main_logger.info("-------new-fit-------")
                    plotwindow = self._create_plot_object()
                else:
                    component = self.peaks_object.peaks.eval(self.peaks_object.params, x=self.x)
                    components = {"Init": component}
                    plotwindow = self._create_plot_object(components=components)
                self.plot_output.clear_output(wait=True)
                display(plotwindow)
            
    def clear_log(self, change=None):
        self.log_output.clear_output()
        
    def experimental(self, change=None):
        with self.log_output:
            print(self.peaks_object)
        
    def set_min(self, what, value):
        w = "w_" + what
        getattr(self, w).min = value
        
    def set_max(self, what, value):
        w = "w_" + what
        getattr(self, w).max = value
        
    def set_value(self, what, value):
        main_logger.info(f"setting value ({value}) to widget (w_{what})")
        w = "w_" + what
        getattr(self, w).value = value
        
    def set_step(self, what, value):
        w = "w_" + what
        getattr(self, w).step = value
        
# ---------------------------------------------------------------------------
visited = {}
options=list(range(10))

def load_cycle(change):
    cycle = change.new
    if cycle in visited.keys():
        c = visited[cycle]
    else:
        cha, volt = my_data.get_ccap(cycle)
        v, dq = ica.dqdv(volt, cha)
        silicon = Silicon(shift=-0.0, max_point=dq.max(), sigma_p1=0.06)
        c = SiliconPeaksFitWidget(silicon, v, dq, f"Cycle {cycle}")
        visited[cycle] = c
    with fit_window:
        fit_window.clear_output(wait=True)
        display(c)

fit_window = widgets.Output()

description = widgets.Label("Select cycle")
sel = widgets.Select(
    options=options,
    value=options[0],
    rows=20,
    disabled=False,
    layout=widgets.Layout(width='70%', height='220px'),
)
sel.observe(load_cycle, 'value')
header = widgets.VBox([description, sel])
out = widgets.HBox([header, fit_window])

handler = log_viewer()
main_logger_out = handler.output

main_logger = logging.getLogger(__name__)
main_logger.addHandler(handler)
main_logger.setLevel(20)   # log at info level.

display(main_logger_out)
display(out)

In [ ]:
#visited[2]

In [ ]:
txt = widgets.Text()
options=list(range(8))
main_title = widgets.Label("Cycle selector")
description = widgets.Label("Select one")
sel = widgets.Select(
    options=options,
    value=options[0],
    rows=10,
    #description='Select one:',
    disabled=False,
    layout=widgets.Layout(width='5%', height='120px'),
)
def set_txt(*args):
    txt.value = str(sel.value)
    
sel.observe(set_txt, 'value')

row1 = widgets.VBox([main_title, txt])
row2 = widgets.VBox([description, sel])
display(widgets.VBox([row1, row2]))

In [ ]:
display(c)

In [ ]:
c.set_value('shift', -0.2)

In [ ]:
c.w_shift.value = 0.001

In [ ]:
c.peaks_object.params.pretty_print()

In [ ]:
result

## Useful functions (OLD)

In [ ]:
def get_widgets(parameters):
    print(parameters)
    

def fix(prefix):
    _pars = p.make_params()
    for k in _pars:
        if k.startswith(prefix):
            p[k].vary = False

def fitplot(v, dq, res, group_title="fit", invert_dq=False, invert_res=False, table=False, width=500, height=500, size=8):
    if invert_dq:
        dq = -dq
        
    i = 1
    if invert_res:
        i = -1
        
    raw = hv.Points((v, dq), label="raw", group=group_title).opts(
        width=width, height=height, size=size, alpha=0.3,
        xlabel="Voltage",
        ylabel="dQ/dv",
    )
    
    prt = {
        "init": hv.Curve((v, i * res.init_fit), group=group_title).opts(alpha=0.5),
        "best": hv.Curve((v, i * res.best_fit), group=group_title),
    }
    
    parts = res.eval_components()

    for key in parts:
        if not key.endswith("Scale"):
            prt[key] = hv.Curve((v, i * parts[key]), group=group_title)
            
    layout = raw * hv.NdOverlay(prt)
    
    if table:
        x = res.best_values
        variables = list(x.keys())
        values = list(x.values())
        lim_min = [res.params[k].min for k in variables]
        lim_max = [res.params[k].max for k in variables]
        vary = [res.params[k].vary for k in variables]
        # expr = [res.params[k].expr for k in variables]
        
        fit_values = {
            "var": variables,
            "val": values,
            "min": lim_min,
            "max": lim_max,
            "vary": vary,
            # "expr": expr,
        }
        
        df_fit_values = pd.DataFrame(fit_values)
        labels_fit_values = hv.Table(df_fit_values).opts(width=700, height=height)

        layout = layout + labels_fit_values
        
    return layout 

## TASKS

#### ipywidgets
Status: Closer (se above)

#### Fitting discharge data
Status: OK

In [ ]:
cha, volt = my_data.get_dcap(2)
v, dq = ica.dqdv(volt, cha)

In [ ]:
silicon = Silicon(shift=-0.1)
graphite = Graphite(shift=-0.03)
dpeaks = CompositeEnsamble()
dpeaks += silicon
dpeaks.add(graphite)

res = dpeaks.fit(-dq, x=v)
layout_d = fitplot(v, dq, res, invert_res=True, group_title="discharge")
layout_d

In [ ]:
cha, volt = my_data.get_dcap(6)
v, dq = ica.dqdv(volt, cha)
dpeaks = CompositeEnsamble(Silicon(shift=-0.1), Graphite(shift=-0.03))
res = dpeaks.fit(-dq, x=v)
#layout_d = fitplot(v, dq, res, invert_res=True, group_title="discharge")
#layout_d

In [ ]:
res.fit_report()

#### Set new parameter values, limits, etc
Status: usable, but not finished

In [ ]:
# setting peak values
# -------------------
# method one (seems a bit convoluted)
#p_si = Silicon()
#p_si.params['Si02sigma'].min = 0
#p_si.peaks.set_param_hint('Si02sigma', min=0.0)

# method two (simpler?)
p_si = Silicon()
p_si.peaks.set_param_hint('Si02sigma', min=2.2)
p_si.peaks.make_params()
p_si.params["Si02sigma"]

In [ ]:
# method two implemented in PeakEnsamble
p_si = Silicon()
print(f"hint: {p_si.peaks.param_hints['Si02sigma']} val: {p_si.params['Si02sigma']}")
p_si.set_param('Si02sigma', minimum=0.02, vary=False)
print(f"hint: {p_si.peaks.param_hints['Si02sigma']} val: {p_si.params['Si02sigma']}")
p_si.reset_peaks()
print(f"hint: {p_si.peaks.param_hints['Si02sigma']} val: {p_si.params['Si02sigma']}")

In [ ]:
prm = "Si02sigma"
step = 0.01
p_si.params[prm].min += step
print(f"hint: {p_si.peaks.param_hints[prm]} val: {p_si.params[prm]}")

# hints are not autmatically updated
p_si.create_hints_from_parameters(prm)
print(f"hint: {p_si.peaks.param_hints[prm]} val: {p_si.params[prm]}")

In [ ]:
# method two implemented in CompositeEnsamble
p_t = CompositeEnsemble(Silicon(), Graphite())
print(f"hint: {p_t.param_hints['Si02sigma']} val: {p_t.params['Si02sigma']}")
p_t.set_param('Si02sigma', minimum=0.02, vary=False)
print(f"hint: {p_t.param_hints['Si02sigma']} val: {p_t.params['Si02sigma']}")
p_t.reset_peaks()
print(f"hint: {p_t.param_hints['Si02sigma']} val: {p_t.params['Si02sigma']}")

In [ ]:
prm = "Si02sigma"
step = 0.01
p_t.params[prm].min += step
print(f"hint: {p_t.param_hints[prm]} val: {p_t.params[prm]}")

# hints are not autmatically updated
p_t.create_hints_from_parameters(prm)
print(f"hint: {p_t.param_hints[prm]} val: {p_t.params[prm]}")

In [ ]:
cha, volt = my_data.get_ccap(4)
v, dq = ica.dqdv(volt, cha)

In [ ]:
p_t.set_param(prm, minimum=0.12)

In [ ]:
# using params_hints
res1 = p_t.fit(dq, x=v)
layout = fitplot(v, dq, res1)
layout

In [ ]:
# using params
res2 = p_t.fit(dq, params=p_t.params, x=v)
layout = fitplot(v, dq, res2)
layout

##### Comments
It seems that there are a multitude of ways to set parameters (params, or param_hints).
When fitting, the default seems to be to use the param_hints. Though, you can also pass the
parameters directly to the `fit` function.

Since `fit` is overridden in our classes, I have the opportunity to fix how it is done.

This is the nethodology chosen:

(to be implemented)

#### Lock the fit-procedure
Make a decition how the `fit` method should be implemented.

Status: Not done

#### Collect fit-results
Status: OK

In [ ]:
def f(n, p, **kwargs):
    # could invent a better name for this function, maybe...
    cha, volt = my_data.get_ccap(n)
    v, dq = ica.dqdv(volt, cha)
    res = p.fit(dq, x=v)
    layout = fitplot(v, dq, res, **kwargs)
    return res, layout

In [ ]:
d = dict()
r = dict()
peaks = CompositeEnsamble(Silicon(crystalline=True), Graphite())
cycle_numbers = my_data.get_cycle_numbers()
for n in cycle_numbers:
    if n == 2:
        print("turning off crystalline peak")
        peaks.ensamble[0].crystalline = False
    try:
        print(f"Fitting cycle {n}", end=" ")
        r[n], d[n] = f(n, peaks, width=300)
        print("-> OK")
    except:
        print("OH NO, Failed!")

In [ ]:
%%opts Curve [width=300]
NdLayout = hv.NdLayout(d, kdims='cycle').cols(4)
NdLayout

In [ ]:
def extract_parameter_values(res):
    x = res.best_values
    variables = list(x.keys())
    
    values = list(x.values())
    lim_min = [res.params[k].min for k in variables]
    lim_max = [res.params[k].max for k in variables]
    vary = [res.params[k].vary for k in variables]
    # expr = [res.params[k].expr for k in variables]

    fit_values = {
        "name": variables,
        "value": values,
        "min": lim_min,
        "max": lim_max,
        "vary": vary,
        # "expr": expr,
    }

    df_fit_values = pd.DataFrame(fit_values)
    df_fit_values.index.name = "parno"
    return df_fit_values

In [ ]:
def combine_parameter_values(r):
    frames = list()
    for n in r:
        df = extract_parameter_values(r[n])
        df["cycle"] = n
        frames.append(df)
    combined = pd.concat(frames).reset_index()
    return combined

In [ ]:
result = combine_parameter_values(r)

In [ ]:
Si02center = result[result.name=="Si02center"]

In [ ]:
curve = hv.Curve((Si02center.cycle, Si02center.value), ("Cycle"), ("Position (V)"))
points = hv.Scatter((Si02center.cycle, Si02center.value), ("Cycle"), ("Position (V)")).opts(size=12)
curve * points

#### Fixing Crystalline peak problem
Status: Usable, but re-setting to crystalline from non-crystalline seems to have some hick-ups

In [ ]:
p_not = CompositeEnsemble(Silicon(), Graphite())
p_is = CompositeEnsemble(Silicon(crystalline=True), Graphite())

In [ ]:
# -getting data
cha, volt = my_data.get_ccap(1)
v, dq = ica.dqdv(volt, cha)

In [ ]:
res_is = p_is.fit(dq, x=v)
layout_cryst = fitplot(v, dq, res_is, group_title="With crystalline peak", table=False)
layout_cryst

In [ ]:
res_is.params

In [ ]:
res_not = p_not.fit(dq, x=v)
layout_not = fitplot(v, dq, res_not, group_title="Without crystalline peak", table=False)
layout_not

In [ ]:
p_not.ensamble[0].crystalline = True
res_not_is = p_not.fit(dq, x=v)
layout_not_is = fitplot(v, dq, res_not_is, group_title="With added crystalline peak", table=False)
layout_not_is

In [ ]:
# -getting data
cha, volt = my_data.get_ccap(2)
v2, dq2 = ica.dqdv(volt, cha)

In [ ]:
p_is.ensamble[0].crystalline = False
# remark! should find out a way to safely transfer or copy the objects if we would like to keep the initial object unchanged
# (this cuould be valuable when using a notebook to prevent problems with excecution order)
res_is2 = p_is.fit(dq2, x=v2)
layout_cryst2 = fitplot(v2, dq2, res_is2, table=False)
layout_cryst2

## OLD STUFF

In [ ]:
# Method one (use the overloaded sum operator to make several peaks)
p1 = Silicon().peaks + Graphite().peaks
pars1 = p1.make_params()

In [ ]:
# Method two (use CompositeEnsamble, the prefered method)
p2 = CompositeEnsamble(Silicon(), Graphite())

In [ ]:
# Method three (consider creating methods for doing this more smoothly)
p3 = CompositeEnsamble()
p3.ensamble.append(Silicon())
p3.ensamble.append(Graphite())
p3._join()

In [ ]:
# Method one and method tow should create the same result (wrt their peaks attribute)
assert p1.param_hints == p2._peaks.param_hints
assert p1.param_hints == p3._peaks.param_hints

In [ ]:
# FITTING CRYSTALLINE PEAK DOES NOT WORK PROPERLY
#p2.ensamble[0].crystalline = True
#p2.ensamble[0].crystalline

In [ ]:
# initial fit
# -getting data
cha, volt = my_data.get_ccap(1)
v, dq = ica.dqdv(volt, cha)

# -fitting
res = p2.fit(dq, x=v)
print("> OK <".center(80, "-"))
# -creating a plot
layout = fitplot(v, dq, res)
layout

In [ ]:
# Information about the fit
# -------------------------
# res.data the measurement data (y-values)
# res.init_fit calculated using the initial peak values (y-values)
# res.best_fit the best fit obtained after fitting (y-values)
# res.params contains fitted parameters object (use e.g. in p2.peaks.eval(params=res.params, x=v))
# res.best_values is a dictionary with the best fitted values (i.e a sub-set of res.params)
# res.fit_report() gives a summary of the fit statistics and values

In [ ]:
def f(n, p, **kwargs):
    cha, volt = my_data.get_ccap(n)
    v, dq = ica.dqdv(volt, cha)
    res = p.fit(dq, x=v)
    layout = fitplot(v, dq, res, **kwargs)
    return layout

In [ ]:
d = dict()
for n in my_data.get_cycle_numbers():
    try:
        print(f"Fitting cycle {n}", end=" ")
        d[n] = f(n, p2, width=300)
        print("-> OK")
    except:
        print("OH NO, Failed!")

In [ ]:
%%opts Curve [width=300]
NdLayout = hv.NdLayout(d, kdims='cycle').cols(4)
NdLayout